# Manufacturer / Aircraft Type Pie Chart

This notebook produces a pie chart of aicraft models for a given manufacturer.

Individual models within a family are grouped:

- e.g. Both A320-214 and A320-232 will appear under the "A320" family
- Aircraft with fewer sightings than the configured threshold are assigned to the "Other" family

In [ ]:
%run ../pathutils.ipynb
%run ../definitions.ipynb
%run ../database.ipynb
%run ../export.ipynb
%run utils.ipynb
%run database.ipynb

In [ ]:
# Sightings threshold below which aircraft are assigned to a generic "Other" group. When set to 1, no models will be assigned to the "other" group
prefix_count_threshold = 1

In [ ]:
import matplotlib.pyplot as plt

def export_pie_chart(export_folder_path, data, manufacturer):
    # Calculate the % of each aircraft type
    group_counts = data["Type"].value_counts()
    group_percent = group_counts / group_counts.sum() * 100

    # Create a new figure
    fig, ax = plt.subplots(figsize=(6, 8))

    # Draw pie chart
    wedges, _ = ax.pie(
        group_percent,
        startangle=90,
        wedgeprops=dict(width=0.5),
        labels=None,
        counterclock=False
    )

    # Format legend labels as Type (XX.X%)"
    legend_labels = [f"{grp} ({pct:.1f}%)" for grp, pct in zip(group_percent.index, group_percent.values)]

    # Add a legend below the chart
    ax.legend(
        wedges,
        legend_labels,
        title="Aircraft Groups",
        loc='lower center',
        bbox_to_anchor=(0.5, -0.2),
        ncol=2,
        fontsize=8
    )

    # Set the title
    plt.title(f"{manufacturer} Aircraft Sightings by Group")
    plt.tight_layout()

    # Export the chart
    export_chart(export_folder_path, f"aircraft-type-pie-chart", "png")

    # Close the plot
    plt.close(fig)


In [ ]:
# Query the data for a list of manufacturers
manufacturers = load_manufacturers()

# Generate a pie chart for each one
for manufacturer in manufacturers:
    # Construct the query
    query = construct_query("aircraft", "manufacturer_sightings.sql", { "MANUFACTURER": manufacturer })

    try:
        # Run the query to load the data - this raises a ValueError if there's no data
        df = query_data("aircraft", query)

        # Extract an aircraft type prefix for each sighting
        df["Type"] = df["Model"].apply(extract_prefix)

        # Identify counts by prefix and use them to identify common prefixes
        prefix_counts = df["Type"].value_counts()
        common_prefixes = prefix_counts[prefix_counts >= prefix_count_threshold].index

        # Replace uncommon prefixes with "Other"
        df["Type"] = df.apply(
            lambda row: row["Type"] if row["Type"] in common_prefixes else "Other",
            axis=1
        )

        # Export the data to excel
        export_folder_path = get_manufacturer_export_folder_path(manufacturer)
        export_to_spreadsheet(export_folder_path, "aircraft-type-pie-chart.xlsx", { "Sightings Data": df })

        # Export the charts
        export_pie_chart(export_folder_path, df, manufacturer)

    except ValueError:
        print(f"WARNING: No data found for manufacturer {manufacturer}")